In [2]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore


cred = credentials.Certificate("firebase_key.json")
firebase_admin.initialize_app(cred)

#db=firestore.client()


In [3]:
data_format = {
    u'behavioral_ratings':   [0, 0, 3, 5, 0, 0, 0, 0, 0, 0],
    u'body_image_ratings':   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    u'grief_ratings':        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    u'relationship_ratings': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    u'depression_ratings':   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    u'physical_ratings':     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
}


#user_ref.set({
#    u'behavioral_ratings':   [1, 0, 0, 0, 4, 0, 0, 0, 2, 0],
#    u'body_image_ratings':   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#    u'grief_ratings':        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#    u'relationship_ratings': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#    u'depression_ratings':   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#    u'physical_ratings':     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#})

In [148]:

from scipy import spatial

def find_links(user_id, category, db):
    user_ref = db.collection(u'userRatings').document(f'{user_id}')
    user = user_ref.get().to_dict()

    max_value =  max(user[f'{category}_ratings'])

    unviewed_links_index = []
    for count, i in enumerate(user[f'{category}_ratings']):
        if i == max_value:
            index = count
        if i != 0:
            pass
        else: 
            unviewed_links_index.append(count)
 
    links_ref = db.collection(u'links').document(f'{category}_links')
    links = links_ref.get().to_dict()
    
    links = [(a,b) for a,b in links_ref.get().to_dict().items()]
    favorite_link = [(a,b) for a,b in links if a.endswith(str(index))][0]
    favorite_link_vector = favorite_link[1][:2] + [-1]

    
    unviewed_links = []

    for i in unviewed_links_index:
        for l in links:
            if l[0].endswith(str(i)):
                unviewed_links.append(l)
    
   
    highest_similarity_link_similarity = 0
    
    for i in unviewed_links:
        unviewed_link_vector = i[1][:2] + [-1]
        similarity = 1 - spatial.distance.cosine(unviewed_link_vector, favorite_link_vector)
        
        if similarity > highest_similarity_link_similarity:
            highest_similarity_link = i
            highest_similarity_link_similarity = similarity
       
    return (highest_similarity_link[1][2], unviewed_links)


https://www.vivapartnership.com/happenings/5-tips-for-coping-with-compulsive-behaviors/


'behavioral_3'

In [101]:
physical_links_ref = db.collection(u'links').document(u'physical_links')
physical_links = physical_links_ref.get().to_dict()

body_image_links_ref = db.collection(u'links').document(u'body_image_links')
body_image_links = body_image_links_ref.get().to_dict()

behavioral_links_ref = db.collection(u'links').document(u'behavioral_links')
behavioral_links = behavioral_links_ref.get().to_dict()


behavioral_links

{'behavioral_9': [1,
  1,
  'https://www.youtube.com/watch?v=8ZhoeSaPF-k&ab_channel=ImprovementPill'],
 'behavioral_3': [0,
  1,
  'https://www.vivapartnership.com/happenings/5-tips-for-coping-with-compulsive-behaviors/'],
 'behavioral_7': [1,
  0,
  'https://www.youtube.com/watch?v=zfKvo1bTi6E&ab_channel=FullPerception'],
 'behavioral_5': [0,
  0,
  'https://www.youtube.com/watch?v=VML6qLgOwKk&ab_channel=TheOutcome'],
 'behavioral_2': [0, 0, 'https://www.healthline.com/health/anger-issues'],
 'behavioral_8': [1,
  1,
  'https://www.youtube.com/watch?v=ytKCcYxUU04&ab_channel=TheSchoolofLife'],
 'behavioral_4': [0,
  1,
  'https://centerstone.org/our-resources/health-wellness/what-are-mood-swings-and-how-to-manage-them/'],
 'behavioral_0': [1,
  0,
  'https://www.verywellmind.com/how-can-i-stop-ocd-thoughts-2510498#:~:text=ask%20a%20doctor.-,Therapy,response%20prevention%20therapy%20(ERP).'],
 'behavioral_1': [0,
  0,
  'https://www.mayoclinic.org/diseases-conditions/anxiety/diagnosis-t

In [103]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd

#1 for link format is video, 0 is article
#1 for link tone is mild, 0 is serious

physical_links_df = pd.DataFrame(physical_links).T
physical_links_df.columns = ['link_format', 'link_tone', 'link']

body_image_links_df = pd.DataFrame(body_image_links).T
body_image_links_df.columns = ['link_format', 'link_tone', 'link']

behavioral_links_df = pd.DataFrame(behavioral_links).T
behavioral_links_df.columns = ['link_format', 'link_tone', 'link']



physical_links_df

,link_format,link_tone,link,dummy_data
physical_7,1,1,https://www.youtube.com/watch?v=hYwXPMvOmuI&ab...,-1
physical_8,1,0,https://www.youtube.com/watch?v=WVootGaWv4M&ab...,-1
physical_4,0,0,https://www.psychologicalscience.org/observer/...,-1
physical_3,0,0,https://www.mayoclinic.org/diseases-conditions...,-1
physical_9,1,0,https://www.youtube.com/watch?v=faonl3-t6MA&ab...,-1
physical_1,0,1,https://www.health.harvard.edu/mind-and-mood/6...,-1
physical_0,1,1,https://www.webmd.com/pain-management/guide/11...,-1
physical_5,1,0,https://www.youtube.com/watch?v=WOlyYbxp_iY&ab...,-1
physical_2,0,1,https://familydoctor.org/deciding-see-doctor/#...,-1
physical_6,1,1,https://www.youtube.com/watch?v=EWnKGxRCNAE&ab...,-1


1.0